In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp -r "/content/drive/MyDrive/deepfake_checkpoints_24f_224" /content/
!cp "/content/drive/MyDrive/deepfake_data.zip" /content/


In [ ]:
!ls /content/


deepfake_checkpoints_24f_224  deepfake_data.zip  drive	sample_data


In [ ]:
!unzip -q /content/deepfake_data.zip -d /content/local_deepfake_data


In [ ]:
INPUT_SHAPE = (24, 224, 224, 3)
N_FRAMES = 24
CHECKPOINT_PATH = '/content/deepfake_checkpoints_24f_224/weights_epoch_{epoch:03d}.weights.h5'
LAST_EPOCH_FILE = '/content/deepfake_checkpoints_24f_224/last_epoch_24f_224.txt'
stage1_epochs = 5
TOTAL_EPOCHS = 20


In [ ]:
# ============================================================
# Environment + Path Configuration (for friend's Colab)
# ============================================================

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.mixed_precision import set_global_policy

try:
    set_global_policy("mixed_float16")
    print("✅ Mixed precision set to 'mixed_float16'.")
except Exception as e:
    print(f"⚠️ Mixed precision unavailable: {e}")

# --- Core config ---
N_FRAMES = 24
BATCH_SIZE = 1
IMG_SIZE = (224, 224)
TOTAL_EPOCHS = 20
STAGE1_EPOCHS = 5
CLASSIFICATION_THRESHOLD = 0.5
INPUT_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], 3)

# --- Friend's Colab paths ---
BASE_PATH = "/content/drive/MyDrive"  # friend's Drive (where you shared the files)
LOCAL_DATA_DIR = "/content/local_deepfake_data"  # extracted dataset folder
CHECKPOINT_DIR = "/content/deepfake_checkpoints_24f_224"  # copied from shared folder

# --- Files and checkpoints ---
LAST_EPOCH_FILE = os.path.join(CHECKPOINT_DIR, "last_epoch_24f_224.txt")
CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "weights_epoch_{epoch:03d}.weights.h5")
BEST_WEIGHTS_PATH = os.path.join(CHECKPOINT_DIR, "best.weights.h5")
TEST_VIDEO_PATH = os.path.join(BASE_PATH, "test_videos", "sample_video.mp4")

print("✅ Paths configured.")
print(f"LOCAL_DATA_DIR: {LOCAL_DATA_DIR}")
print(f"CHECKPOINT_DIR: {CHECKPOINT_DIR}")


✅ Mixed precision set to 'mixed_float16'.
✅ Paths configured.
LOCAL_DATA_DIR: /content/local_deepfake_data
CHECKPOINT_DIR: /content/deepfake_checkpoints_24f_224


In [ ]:
# ============================================================
# Build Model (exactly as used in your first 7 epochs)
# ============================================================

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    GRU, Dense, TimeDistributed, Input,
    GlobalAveragePooling2D, Dropout
)
from tensorflow.keras.applications import MobileNetV2

def build_model(input_shape, n_frames):
    """
    Builds the MobileNetV2 + GRU hybrid model for deepfake detection.
    Matches exactly the architecture used during the first 7 epochs.
    """
    # Base CNN for spatial feature extraction
    base = MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=input_shape,
        alpha=0.75
    )
    x = GlobalAveragePooling2D()(base.output)
    feature_extractor = Model(base.input, x)

    # Temporal model for sequence of frames
    seq_input = Input(shape=(n_frames, *input_shape))
    encoded = TimeDistributed(feature_extractor)(seq_input)
    gru = GRU(32)(encoded)
    d = Dropout(0.5)(gru)
    x = Dense(32, activation="relu")(d)
    x = Dropout(0.5)(x)
    out = Dense(1, activation="sigmoid", dtype="float32")(x)

    model = Model(seq_input, out)
    return model, base


In [ ]:
# ============================================================
# AFTER UNZIPPING: Resume-safe 2-stage training on friend's Colab
# ============================================================

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# ============================================================
# 1️⃣ Data Generator (exact same version used during first 7 epochs)
# ============================================================

class VideoSequenceGenerator(Sequence):
    def __init__(self, metadata, batch_size, n_frames, img_size, data_path, shuffle=True):
        self.metadata = metadata
        self.batch_size = batch_size
        self.n_frames = n_frames
        self.img_size = img_size
        self.data_path = data_path
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.metadata) / self.batch_size))

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.metadata))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_meta = [self.metadata[k] for k in indexes]
        X = np.empty((self.batch_size, self.n_frames, *self.img_size, 3), dtype=np.float32)
        y = np.empty((self.batch_size, 1), dtype=np.float32)
        for i, (vid_rel, label) in enumerate(batch_meta):
            abs_path = os.path.join(self.data_path, vid_rel)
            X[i,], y[i,] = self._load_sample(abs_path, label)
        return X, y

    def _load_sample(self, path, label):
        try:
            frames = np.load(path, allow_pickle=False)
            frames = preprocess_input(frames.astype(np.float32))
            y_data = np.array([float(label)], dtype=np.float32)
            return frames, y_data
        except Exception as e:
            print(f"⚠️ Error loading {path}: {e}")
            X_fail = np.zeros((self.n_frames, *self.img_size, 3), dtype=np.float32)
            y_fail = np.array([0.0], dtype=np.float32)
            return X_fail, y_fail


# ============================================================
# 2️⃣ Main Training Function (resume-safe, 2-stage)
# ============================================================

def train_after_unzip():
    # 1) Load metadata
    train_meta = np.load(os.path.join(LOCAL_DATA_DIR, "train_metadata.npy"), allow_pickle=True)
    val_meta   = np.load(os.path.join(LOCAL_DATA_DIR, "val_metadata.npy"), allow_pickle=True)
    print(f"📁 Train samples: {len(train_meta)}, Val samples: {len(val_meta)}")

    # 2) Create generators
    train_gen = VideoSequenceGenerator(train_meta, BATCH_SIZE, N_FRAMES, IMG_SIZE, LOCAL_DATA_DIR, shuffle=True)
    val_gen   = VideoSequenceGenerator(val_meta,   BATCH_SIZE, N_FRAMES, IMG_SIZE, LOCAL_DATA_DIR, shuffle=False)

    # 3) Build model (uses your original architecture)
    model, base = build_model(INPUT_SHAPE, N_FRAMES)
    model.summary()

    # 4) Check for existing progress
    initial_epoch = 0
    if os.path.exists(LAST_EPOCH_FILE):
        try:
            with open(LAST_EPOCH_FILE, "r") as f:
                txt = f.read().strip()
            if txt.isdigit():
                initial_epoch = int(txt)
        except Exception as e:
            print(f"⚠️ Could not parse LAST_EPOCH_FILE: {e}")
            initial_epoch = 0

    # 5) Load existing weights safely
    if initial_epoch > 0:
        resume_path = CHECKPOINT_PATH.format(epoch=initial_epoch)
        if os.path.exists(resume_path):
            try:
                model.load_weights(resume_path)
                print(f"✅ Loaded weights from {resume_path}")
            except Exception as e:
                print(f"⚠️ Strict load failed: {e}")
                try:
                    model.load_weights(resume_path, weights_only=True)
                    print("✅ Loaded matching layers only (partial load).")
                except Exception as e2:
                    print(f"❌ Partial load failed: {e2}. Starting from scratch.")
                    initial_epoch = 0
        else:
            print("⚠️ No weight file found for resume. Starting fresh.")
            initial_epoch = 0
    else:
        print("🆕 Starting training from epoch 1.")

    # ============================================================
    # 6️⃣ Define Callbacks
    # ============================================================

    class EpochSaverCallback(Callback):
        def on_epoch_end(self, epoch, logs=None):
            e = epoch + 1
            weight_path = CHECKPOINT_PATH.format(epoch=e)
            self.model.save_weights(weight_path)
            with open(LAST_EPOCH_FILE, "w") as f:
                f.write(str(e))
            print(f"\n💾 Saved checkpoint {weight_path} (epoch {e})")

    early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    ckpt_best = ModelCheckpoint(
        BEST_WEIGHTS_PATH, monitor="val_loss", mode="min",
        save_best_only=True, save_weights_only=True, verbose=1
    )

    callbacks = [EpochSaverCallback(), early_stop, ckpt_best]

    # ============================================================
    # 7️⃣ Stage 1 – Warm-Up
    # ============================================================

    last_completed_epoch = initial_epoch

    if initial_epoch < STAGE1_EPOCHS:
        print(f"\n--- Stage 1: Freeze MobileNetV2 (epochs {initial_epoch + 1} → {STAGE1_EPOCHS}) ---")
        for l in base.layers:
            l.trainable = False

        model.compile(
            optimizer=tf.keras.optimizers.Adam(1e-3),
            loss="binary_crossentropy",
            metrics=["accuracy"]
        )

        hist1 = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=STAGE1_EPOCHS,
            initial_epoch=initial_epoch,
            callbacks=callbacks
        )

        if hasattr(hist1, "epoch") and len(hist1.epoch) > 0:
            last_completed_epoch = hist1.epoch[-1] + 1
        else:
            last_completed_epoch = initial_epoch
        initial_epoch = last_completed_epoch
        print(f"✅ Stage 1 completed. Now resuming from epoch {initial_epoch}.")

    # ============================================================
    # 8️⃣ Stage 2 – Fine-Tuning
    # ============================================================

    if initial_epoch < TOTAL_EPOCHS:
        print(f"\n--- Stage 2: Fine-tune MobileNetV2 (epochs {initial_epoch + 1} → {TOTAL_EPOCHS}) ---")
        for l in base.layers:
            l.trainable = True

        model.compile(
            optimizer=tf.keras.optimizers.Adam(1e-5),
            loss="binary_crossentropy",
            metrics=["accuracy"]
        )

        hist2 = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=TOTAL_EPOCHS,
            initial_epoch=initial_epoch,
            callbacks=callbacks
        )

        if hasattr(hist2, "epoch") and len(hist2.epoch) > 0:
            last_completed_epoch = hist2.epoch[-1] + 1
        else:
            last_completed_epoch = initial_epoch
        print(f"✅ Stage 2 complete. Trained up to epoch {last_completed_epoch}.")
    else:
        last_completed_epoch = initial_epoch

    # ============================================================
    # 9️⃣ Optional: Run test prediction
    # ============================================================

    if last_completed_epoch > 0:
        try:
            predict_video(model, last_completed_epoch)
        except Exception as e:
            print(f"⚠️ Prediction skipped: {e}")


# ============================================================
# Run Training
# ============================================================

train_after_unzip()


📁 Train samples: 2340, Val samples: 585


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 24, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 24, 1280)       │     1,382,064 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │       126,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,509,297 (5.76 MB)

 Trainable params: 1,482,657 (5.66 MB)

 Non-trainable params: 26,640 (104.06 KB)

✅ Loaded weights from /content/deepfake_checkpoints_24f_224/weights_epoch_007.weights.h5

--- Stage 2: Fine-tune MobileNetV2 (epochs 8 → 20) ---
Epoch 8/20
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 0s 706ms/step - accuracy: 0.6347 - loss: 0.6520
💾 Saved checkpoint /content/deepfake_checkpoints_24f_224/weights_epoch_008.weights.h5 (epoch 8)

Epoch 8: val_loss improved from inf to 0.67557, saving model to /content/deepfake_checkpoints_24f_224/best.weights.h5
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 2111s 761ms/step - accuracy: 0.6347 - loss: 0.6520 - val_accuracy: 0.6000 - val_loss: 0.6756
Epoch 9/20
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 0s 705ms/step - accuracy: 0.7279 - loss: 0.5838
💾 Saved checkpoint /content/deepfake_checkpoints_24f_224/weights_epoch_009.weights.h5 (epoch 9)

Epoch 9: val_loss did not improve from 0.67557
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 1739s 743ms/step - accuracy: 0.7279 - loss: 0.5838 - val_accuracy: 0.5675 - val_loss: 0.6881
Epoch 10/20
2340/2340 ━━━━━━━━━━━━━━━━━━━━ 0s 704ms/step - accuracy:

KeyboardInterrupt: 